## 问题定义

本问题为求解2D Helmholtz方程。其形式可以参考案例18中的PDE定义。


### 求解目标

要求求解给定区域内的Helmholtz方程

## Adaptive Activation Function

在一般激活函数的基础上加入了可学习参数从而能够调整激活函数的slope。


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter

In [3]:
from sympy import Symbol, pi, sin

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.eq.pdes.wave_equation import HelmholtzEquation

In [4]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 20000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:

### 定义必要组件

#### Geo

In [5]:
x, y = Symbol("x"), Symbol("y")
height = 2
width = 2
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

#### PDE

In [6]:
wave = HelmholtzEquation(u="u", k=1.0, dim=2)  # 亥姆霍兹方程

#### Model

In [7]:
# 定义简单的全连接网络
# 输入为空间坐标和
# 输出为E_z(x, y)
wave_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u")],
    cfg=cfg.arch.fully_connected,
)

#### Node

In [8]:
nodes = wave.make_nodes() + [wave_net.make_node(name="wave_network")]

#### Domain

In [9]:
domain = Domain()

#### ABC边界

In [10]:
wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0},
    batch_size=cfg.batch_size.wall,
)
domain.add_constraint(wall, "wall")

#### PDE约束

In [11]:
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "helmholtz": -(
            -((pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            - ((4 * pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            + 1 * sin(pi * x) * sin(4 * pi * y)
        )
    },  # 非齐次形式
    batch_size=cfg.batch_size.interior,
    bounds={x: (-width / 2, width / 2), y: (-height / 2, height / 2)},
    lambda_weighting={
        "helmholtz": Symbol("sdf"),
    },
)
domain.add_constraint(interior, "interior")

#### 验证器以及其他必要组件

In [12]:
file_path = "openfoam/helmholtz.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"x": "x", "y": "y", "z": "u"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u"]
    }

    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [13]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [14]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [15]:
slv.solve()

2024-03-03 09:31:58,104 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-03 09:31:58,114 - INFO - attempting to restore from: /workspace/24_2D_Helmholtz_AdaptiveActivationFunc/outputs
2024-03-03 09:31:58,115 - WARNING - optimizer checkpoint not found
2024-03-03 09:31:58,116 - WARNING - model wave_network.0.pth not found
2024-03-03 09:31:59,416 - INFO - [step:          0] record constraint batch time:  6.533e-02s
2024-03-03 09:32:11,590 - INFO - [step:          0] record validators time:  1.217e+01s
2024-03-03 09:32:11,686 - INFO - [step:          0] saved checkpoint to /workspace/24_2D_Helmholtz_AdaptiveActivationFunc/outputs
2024-03-03 09:32:11,689 - INFO - [step:          0] loss:  9.836e+03
2024-03-03 09:32:13,178 - INFO - Attempting cuda graph building, this may take a bit...
2024-03-03 09:32:25,660 - INFO - [step:        100] loss:  4.192e+03, time/iteration:  1.397e+02 ms
2024-0

2024-03-03 09:36:08,094 - INFO - [step:       6000] saved checkpoint to /workspace/24_2D_Helmholtz_AdaptiveActivationFunc/outputs
2024-03-03 09:36:08,096 - INFO - [step:       6000] loss:  1.581e-01, time/iteration:  1.437e+02 ms
2024-03-03 09:36:10,538 - INFO - [step:       6100] loss:  2.655e-01, time/iteration:  2.440e+01 ms
2024-03-03 09:36:12,949 - INFO - [step:       6200] loss:  6.903e-02, time/iteration:  2.410e+01 ms
2024-03-03 09:36:15,362 - INFO - [step:       6300] loss:  1.777e-01, time/iteration:  2.412e+01 ms
2024-03-03 09:36:17,809 - INFO - [step:       6400] loss:  4.532e+00, time/iteration:  2.445e+01 ms
2024-03-03 09:36:20,261 - INFO - [step:       6500] loss:  2.188e-01, time/iteration:  2.452e+01 ms
2024-03-03 09:36:22,689 - INFO - [step:       6600] loss:  1.104e-01, time/iteration:  2.426e+01 ms
2024-03-03 09:36:25,119 - INFO - [step:       6700] loss:  2.500e-01, time/iteration:  2.428e+01 ms
2024-03-03 09:36:27,549 - INFO - [step:       6800] loss:  9.569e-02, 

2024-03-03 09:39:58,485 - INFO - [step:      12500] loss:  7.134e-02, time/iteration:  2.454e+01 ms
2024-03-03 09:40:00,943 - INFO - [step:      12600] loss:  3.240e-02, time/iteration:  2.457e+01 ms
2024-03-03 09:40:03,401 - INFO - [step:      12700] loss:  9.160e-02, time/iteration:  2.456e+01 ms
2024-03-03 09:40:05,856 - INFO - [step:      12800] loss:  3.176e-02, time/iteration:  2.454e+01 ms
2024-03-03 09:40:08,304 - INFO - [step:      12900] loss:  3.149e-02, time/iteration:  2.446e+01 ms
2024-03-03 09:40:23,211 - INFO - [step:      13000] record validators time:  1.190e+01s
2024-03-03 09:40:23,240 - INFO - [step:      13000] saved checkpoint to /workspace/24_2D_Helmholtz_AdaptiveActivationFunc/outputs
2024-03-03 09:40:23,242 - INFO - [step:      13000] loss:  4.423e-02, time/iteration:  1.494e+02 ms
2024-03-03 09:40:25,690 - INFO - [step:      13100] loss:  3.238e-02, time/iteration:  2.447e+01 ms
2024-03-03 09:40:28,166 - INFO - [step:      13200] loss:  3.171e-02, time/iterati

2024-03-03 09:44:00,289 - INFO - [step:      19000] loss:  2.833e-02, time/iteration:  1.359e+02 ms
2024-03-03 09:44:02,755 - INFO - [step:      19100] loss:  2.148e-02, time/iteration:  2.464e+01 ms
2024-03-03 09:44:05,383 - INFO - [step:      19200] loss:  2.146e-02, time/iteration:  2.627e+01 ms
2024-03-03 09:44:07,915 - INFO - [step:      19300] loss:  2.474e-02, time/iteration:  2.530e+01 ms
2024-03-03 09:44:10,409 - INFO - [step:      19400] loss:  2.450e-02, time/iteration:  2.492e+01 ms
2024-03-03 09:44:12,795 - INFO - [step:      19500] loss:  2.264e-02, time/iteration:  2.383e+01 ms
2024-03-03 09:44:15,237 - INFO - [step:      19600] loss:  2.362e-02, time/iteration:  2.442e+01 ms
2024-03-03 09:44:17,753 - INFO - [step:      19700] loss:  2.372e-02, time/iteration:  2.514e+01 ms
2024-03-03 09:44:20,265 - INFO - [step:      19800] loss:  2.307e-02, time/iteration:  2.509e+01 ms
2024-03-03 09:44:22,763 - INFO - [step:      19900] loss:  2.334e-02, time/iteration:  2.496e+01 ms


### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/validators/validator_u.png)